In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

In [11]:
# df = pd.read_csv(r'C:\Users\hameedbf\Documents\Python\emi_prediction\emi_prediction_dataset.csv')
df = pd.read_csv(r'C:\emi_prediction\emi_prediction_dataset.csv')
df

C:\Users\admin\AppData\Local\Temp\ipykernel_15772\1403777051.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\emi_prediction\emi_prediction_dataset.csv')


,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
0,38.0,Female,Married,Professional,82600.0,Private,0.9,Mid-size,Rented,20000.0,...,Yes,23700.0,660.0,303200.0,70200.0,Personal Loan EMI,850000.0,15,Not_Eligible,500.0
1,38.0,Female,Married,Graduate,21500.0,Private,7.0,MNC,Family,0.0,...,Yes,4100.0,714.0,92500.0,26900.0,E-commerce Shopping EMI,128000.0,19,Not_Eligible,700.0
2,38.0,Male,Married,Professional,86100.0,Private,5.8,Startup,Own,0.0,...,No,0.0,650.0,672100.0,324200.0,Education EMI,306000.0,16,Eligible,27775.0
3,58.0,Female,Married,High School,66800.0,Private,2.2,Mid-size,Own,0.0,...,No,0.0,685.0,440900.0,178100.0,Vehicle EMI,304000.0,83,Eligible,16170.0
4,48.0,Female,Married,Professional,57300.0,Private,3.4,Mid-size,Family,0.0,...,No,0.0,770.0,97300.0,28200.0,Home Appliances EMI,252000.0,7,Not_Eligible,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404795,27.0,Male,Married,Graduate,32400.0,Private,5.0,Large Indian,Rented,10400.0,...,No,0.0,649.0,62000.0,32600.0,Personal Loan EMI,506000.0,47,Not_Eligible,500.0
404796,38.0,Male,Married,Post Graduate,49200.0,Private,1.9,MNC,Own,0.0,...,No,0.0,712.0,142200.0,38100.0,Personal Loan EMI,708000.0,33,Not_Eligible,5200.0
404797,32.0,Male,Single,Graduate,25700.0,Private,3.2,MNC,Rented,6300.0,...,No,0.0,676.0,191600.0,39700.0,Home Appliances EMI,93000.0,21,High_Risk,5665.0
404798,48.0,Male,Married,Graduate,47200.0,Private,3.0,MNC,Own,0.0,...,No,0.0,784.0,170400.0,45600.0,Home Appliances EMI,144000.0,36,Eligible,14460.0


In [17]:
df.columns

Index(['age', 'gender', 'marital_status', 'education', 'monthly_salary',
       'employment_type', 'years_of_employment', 'company_type', 'house_type',
       'monthly_rent', 'family_size', 'dependents', 'school_fees',
       'college_fees', 'travel_expenses', 'groceries_utilities',
       'other_monthly_expenses', 'existing_loans', 'current_emi_amount',
       'credit_score', 'bank_balance', 'emergency_fund', 'emi_scenario',
       'requested_amount', 'requested_tenure', 'emi_eligibility',
       'max_monthly_emi'],
      dtype='object')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404800 entries, 0 to 404799
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   age                     404800 non-null  object 
 1   gender                  404800 non-null  object 
 2   marital_status          404800 non-null  object 
 3   education               402396 non-null  object 
 4   monthly_salary          404800 non-null  object 
 5   employment_type         404800 non-null  object 
 6   years_of_employment     404800 non-null  float64
 7   company_type            404800 non-null  object 
 8   house_type              404800 non-null  object 
 9   monthly_rent            402374 non-null  float64
 10  family_size             404800 non-null  int64  
 11  dependents              404800 non-null  int64  
 12  school_fees             404800 non-null  float64
 13  college_fees            404800 non-null  float64
 14  travel_expenses     

Checking row values by going through each column

In [3]:
def age_type(age):
    updated_age = ""
    j=0
    for i in str(age):
        j+=1
        if i.isdigit():
            updated_age += i
        if j==2:
            break
    return int(updated_age)
df['age'] = df['age'].apply(age_type)

In [4]:
df['gender'] = df['gender'].replace({'MALE':'Male', 'M': 'Male', 'male':'Male',
                                     'F':'Female', 'female':'Female', 'FEMALE':'Female'  })

In [5]:
df['bank_balance'] = (
    df['bank_balance']
    .astype(str)                      # convert all to string for cleaning
    .str.replace(r'\.0\.0$', '.00', regex=True)  # fix ".0.0" to ".00"
    .str.replace(r'\.0$', '.00', regex=True)     # ensure all end with .00
)

# Convert back to numeric if you need to plot or calculate
df['bank_balance'] = pd.to_numeric(df['bank_balance'], errors='coerce')


In [6]:
df['monthly_salary'] = (
    df['monthly_salary']
    .astype(str)                      # convert all to string for cleaning
    .str.replace(r'\.0\.0$', '.00', regex=True)  # fix ".0.0" to ".00"
    .str.replace(r'\.0$', '.00', regex=True)     # ensure all end with .00
)
df['monthly_salary'] = pd.to_numeric(df['monthly_salary'], errors='coerce')

In [7]:
df.dropna(inplace=True)

In [8]:
df.describe()

,age,monthly_salary,years_of_employment,monthly_rent,family_size,dependents,school_fees,college_fees,travel_expenses,groceries_utilities,other_monthly_expenses,current_emi_amount,credit_score,bank_balance,emergency_fund,requested_amount,requested_tenure,max_monthly_emi
count,392899.000000,392899.000000,392899.000000,392899.000000,392899.00000,392899.00000,392899.000000,392899.000000,392899.000000,392899.000000,392899.000000,392899.000000,392899.000000,3.928990e+05,392899.000000,3.928990e+05,392899.000000,392899.000000
mean,38.877488,59498.624547,5.368834,5827.342918,2.94070,1.94070,4625.387695,4066.852041,5687.161586,12805.942494,7120.359940,4541.927060,700.867241,2.417099e+05,96771.688144,3.709153e+05,29.127822,6765.939973
std,9.300407,43343.845798,6.084231,8646.245400,1.07502,1.07502,5061.094643,7320.204884,3392.866112,6993.387321,4511.036034,7034.481551,88.499348,1.832477e+05,81358.605124,3.451781e+05,18.096444,7746.599576
min,26.000000,3967.000000,0.500000,0.000000,1.00000,0.00000,0.000000,0.000000,600.000000,1800.000000,600.000000,0.000000,0.000000,6.100000e+03,1400.000000,1.000000e+04,3.000000,500.000000
25%,32.000000,35400.000000,1.200000,0.000000,2.00000,1.00000,0.000000,0.000000,3200.000000,7700.000000,3800.000000,0.000000,654.000000,1.044000e+05,38400.000000,1.240000e+05,15.000000,500.000000
50%,38.000000,51700.000000,3.200000,0.000000,3.00000,2.00000,3100.000000,0.000000,4900.000000,11400.000000,6000.000000,0.000000,701.000000,1.960000e+05,74000.000000,2.370000e+05,25.000000,4217.400000
75%,48.000000,73000.000000,7.200000,10600.000000,4.00000,3.00000,9000.000000,6500.000000,7400.000000,16400.000000,9300.000000,8000.000000,748.000000,3.313000e+05,130600.000000,4.950000e+05,40.000000,9792.000000
max,59.000000,499970.000000,36.000000,80000.000000,5.00000,4.00000,15000.000000,25000.000000,30300.000000,71200.000000,42900.000000,56300.000000,1200.000000,1.717300e+06,891500.000000,1.500000e+06,84.000000,91040.400000


In [24]:
# Select only numeric columns
numeric_df = df.select_dtypes(include=['int64', 'float64'])

# Compute correlation matrix
corr_matrix = numeric_df.corr()

# Create a heatmap
fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    title='Correlation Heatmap (Numeric Features)'
)

fig.update_layout(width=1000, height=800)
fig.show()


In [9]:
# Data Cleaning completed and Extracting to CSV
df.to_csv("cleaned_emi_dataset.csv", index=False)

In [22]:
#Using Random forest to predict max_monthly_emi with HyperParameter Tuning
X = df.drop('max_monthly_emi', axis=1)
y = df['max_monthly_emi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)

In [23]:
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [31]:
y_pred = rf.predict(X_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- max_monthly_emi
Feature names seen at fit time, yet now missing:
- emi_eligibility


In [ ]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("=== Random Forest Regressor Performance ===")
print(f"MAE  : {mae:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"R²   : {r2:.2f}")


=== Random Forest Regressor Performance ===
MAE  : 829.37
RMSE : 1444.10
R²   : 0.96


In [28]:
# Combine actual and predicted values into one DataFrame
comparison = pd.DataFrame({
    'Actual_EMI': y_test.values,
    'Predicted_EMI': y_pred
})

# Display top 10 (or all if you prefer)
print(comparison.head(20))


    Actual_EMI  Predicted_EMI
0      5445.00      5061.1300
1      5665.00      4699.4490
2       500.00       504.5000
3      6195.20      5131.3070
4      3963.96      3919.1614
5      1134.00      1097.8658
6     16080.00     14606.7530
7      7650.00      7916.4180
8     24400.00     23980.2880
9     11814.00     10085.3760
10     7177.50      8936.7900
11      500.00       703.5470
12     2072.00      2048.1173
13     8750.00      9083.0700
14     7260.00      8618.9400
15     5227.20      9427.6280
16    17960.00     19499.0940
17     6809.60      6921.2136
18      500.00       500.0000
19    11286.00     12210.7580


In [58]:
#Random Forest Classifier for emi_eligibility without HyperParameter Tuning
X = df.drop('emi_eligibility', axis=1)
y = df['emi_eligibility']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')

In [59]:
rf_clf.fit(X_train, y_train)

,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [60]:
y_pred = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95     14283
           1       0.86      0.04      0.07      3454
           2       0.96      1.00      0.98     60843

    accuracy                           0.95     78580
   macro avg       0.91      0.67      0.67     78580
weighted avg       0.95      0.95      0.93     78580



In [61]:
print(confusion_matrix(y_test, y_pred))

[[13796    17   470]
 [  929   131  2394]
 [   50     5 60788]]


In [63]:
# y_pred = model.predict(X_test)  # numeric predictions (0,1,2)
y_pred_labels = le.inverse_transform(y_pred)

# Similarly, for test labels
y_test_labels = le.inverse_transform(y_test)
comparison = pd.DataFrame({
    'Actual': y_test_labels,
    'Predicted': y_pred_labels
})

print(comparison.head(30))

          Actual     Predicted
0   Not_Eligible  Not_Eligible
1       Eligible      Eligible
2   Not_Eligible  Not_Eligible
3   Not_Eligible  Not_Eligible
4   Not_Eligible  Not_Eligible
5   Not_Eligible  Not_Eligible
6   Not_Eligible  Not_Eligible
7   Not_Eligible  Not_Eligible
8   Not_Eligible  Not_Eligible
9   Not_Eligible  Not_Eligible
10  Not_Eligible  Not_Eligible
11  Not_Eligible  Not_Eligible
12  Not_Eligible  Not_Eligible
13  Not_Eligible  Not_Eligible
14  Not_Eligible  Not_Eligible
15  Not_Eligible  Not_Eligible
16  Not_Eligible  Not_Eligible
17  Not_Eligible  Not_Eligible
18  Not_Eligible  Not_Eligible
19  Not_Eligible  Not_Eligible
20  Not_Eligible  Not_Eligible
21  Not_Eligible  Not_Eligible
22  Not_Eligible  Not_Eligible
23  Not_Eligible  Not_Eligible
24  Not_Eligible  Not_Eligible
25  Not_Eligible  Not_Eligible
26     High_Risk  Not_Eligible
27  Not_Eligible  Not_Eligible
28  Not_Eligible  Not_Eligible
29      Eligible      Eligible


In [64]:
#Linear Regression Model to predict emi_eligibility
from sklearn.linear_model import LinearRegression

In [68]:
X = df.drop('max_monthly_emi', axis=1)
y = df['max_monthly_emi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lr = LinearRegression()

In [69]:
lr.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [73]:
y_pred = lr.predict(X_test)
pd.DataFrame({"Actual":y_test.values, "Predicted":y_pred})

,Actual,Predicted
0,500.0,662.622373
1,37422.0,26130.744198
2,4080.0,6629.652840
3,2485.0,2917.542988
4,22500.0,16875.085657
...,...,...
78575,500.0,-2335.442241
78576,5450.0,8393.102680
78577,500.0,-2551.093929
78578,1830.4,6950.233367


In [74]:
print ("mse:",mean_squared_error(y_test, y_pred))
print ("rmse:",np.sqrt(mean_squared_error(y_test, y_pred)))
print ("r2_score:",r2_score(y_test, y_pred))
print ("mape:",mean_absolute_percentage_error(y_test, y_pred))

mse: 14823741.5901075
rmse: 3850.1612420920114
r2_score: 0.7511583106343509
mape: 1.6836570141300935


In [86]:
#Logistics Regression Model to predict emi_eligibility
from sklearn.linear_model import LogisticRegression
X = df.drop("emi_eligibility", axis=1)
y = df["emi_eligibility"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
log = LogisticRegression(solver='saga', penalty='l2', C=0.1, class_weight='balanced')
log.fit(X_train, y_train)

c:\Users\hameedbf\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



,penalty,'l2'
,dual,False
,tol,0.0001
,C,0.1
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'saga'
,max_iter,100
,multi_class,'deprecated'


In [87]:
y_pred = log.predict(X_test)

In [88]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.75      0.77     14283
           1       0.15      0.61      0.23      3454
           2       0.99      0.82      0.90     60843

    accuracy                           0.80     78580
   macro avg       0.64      0.73      0.63     78580
weighted avg       0.91      0.80      0.84     78580



In [108]:
# MLflow + Linear Regression without HyperParameter Tuning
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import pandas as pd
import numpy as np

# --- Features and target ---
X = df.drop('max_monthly_emi', axis=1)
y = df['max_monthly_emi']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set MLflow experiment
mlflow.set_experiment("EMI Prediction Experiment")

with mlflow.start_run(run_name="Linear Regression without Tuning"):  # start a new MLflow run
    
    # --- Train Linear Regression ---
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    # --- Predict ---
    y_pred = lr.predict(X_test)
    
    # --- Calculate metrics ---
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    # --- Log parameters (Linear Regression has few parameters, just for reference) ---
    mlflow.log_param("model_type", "Linear Regression with no HyperParameter Tuning")
    
    # --- Log metrics ---
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2_score", r2)
    mlflow.log_metric("mape", mape)
    
    # --- Log model ---
    mlflow.sklearn.log_model(lr, "linear_regression_model")
    
    # --- Display metrics in notebook ---
    print(f"MSE   : {mse:.2f}")
    print(f"RMSE  : {rmse:.2f}")
    print(f"R²    : {r2:.2f}")
    print(f"MAPE  : {mape:.4f}")

# --- Optional: compare Actual vs Predicted ---
comparison = pd.DataFrame({
    "Actual": y_test.values,
    "Predicted": y_pred
})
print(comparison.head(10))


2025/10/16 15:47:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 15:47:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MSE   : 14823741.59
RMSE  : 3850.16
R²    : 0.75
MAPE  : 1.6837
    Actual     Predicted
0    500.0    662.622373
1  37422.0  26130.744198
2   4080.0   6629.652840
3   2485.0   2917.542988
4  22500.0  16875.085657
5    500.0   7621.962966
6   7459.2   8286.044021
7    500.0    105.564101
8    500.0    876.261592
9   2240.0   3413.479618


In [109]:
# MLflow + Linear Regression with HyperParameter Tuning - Gradient Descent, Regularization
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
# --- Features and target ---
X = df.drop('max_monthly_emi', axis=1)
y = df['max_monthly_emi']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set MLflow experiment
mlflow.set_experiment("EMI Prediction Experiment")

with mlflow.start_run(run_name="Linear Regression with HyperParameter Tuning"):  # start a new MLflow run

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # --- Train Linear Regression ---
    lr_scaled = SGDRegressor(learning_rate= "invscaling")
    lr_scaled.fit(X_train_scaled,y_train)
    
    # --- Predict ---
    y_pred_scaled = lr_scaled.predict(X_test_scaled)
    
    # --- Calculate metrics ---
    mse = mean_squared_error(y_test, y_pred_scaled)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred_scaled)
    mape = mean_absolute_percentage_error(y_test, y_pred_scaled)
    
    # --- Log parameters (Linear Regression has few parameters, just for reference) ---
    mlflow.log_param("model_type", "Linear Regression with no HyperParameter Tuning")
    
    # --- Log metrics ---
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2_score", r2)
    mlflow.log_metric("mape", mape)
    
    # --- Log model ---
    mlflow.sklearn.log_model(lr, "linear_regression_model")
    
    # --- Display metrics in notebook ---
    print(f"MSE   : {mse:.2f}")
    print(f"RMSE  : {rmse:.2f}")
    print(f"R²    : {r2:.2f}")
    print(f"MAPE  : {mape:.4f}")

# --- Optional: compare Actual vs Predicted ---
comparison = pd.DataFrame({
    "Actual": y_test.values,
    "Predicted": y_pred_scaled
})
print(comparison.head(10))

2025/10/16 15:48:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 15:48:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MSE   : 14865310.35
RMSE  : 3855.56
R²    : 0.75
MAPE  : 1.6803
    Actual     Predicted
0    500.0    626.364121
1  37422.0  25903.955006
2   4080.0   6709.850438
3   2485.0   2967.199638
4  22500.0  16828.040364
5    500.0   7455.147833
6   7459.2   8278.336354
7    500.0   -183.760360
8    500.0    728.208345
9   2240.0   3412.781960


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404800 entries, 0 to 404799
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   age                     404800 non-null  object 
 1   gender                  404800 non-null  object 
 2   marital_status          404800 non-null  object 
 3   education               402396 non-null  object 
 4   monthly_salary          404800 non-null  object 
 5   employment_type         404800 non-null  object 
 6   years_of_employment     404800 non-null  float64
 7   company_type            404800 non-null  object 
 8   house_type              404800 non-null  object 
 9   monthly_rent            402374 non-null  float64
 10  family_size             404800 non-null  int64  
 11  dependents              404800 non-null  int64  
 12  school_fees             404800 non-null  float64
 13  college_fees            404800 non-null  float64
 14  travel_expenses     